In [ ]:
from mpasanalysis import *
%matplotlib inline

In [ ]:
# flags
save_fig = False

In [ ]:
# get hostname
hostname = os.getenv('HOSTNAME')
print('Running on machine {}'.format(hostname))

In [ ]:
# set paths
if 'theta' in hostname:
    data_root = '/projects/ClimateEnergy_3/azamatm/E3SM_simulations/theta.20180906.branch_noCNT.A_WCYCL1950S_CMIP6_HR.ne120_oRRS18v3_ICG'
    fig_root = '/home/qingli/work/e3sm_res_cmp/figures/high_res'
    rst_file = data_root+'/run/mpaso.rst.0051-01-01_00000.nc'
    mon_file = data_root+'/run/mpaso.hist.am.timeSeriesStatsMonthly.0051-01-01.nc'
elif 'edison' in hostname:
    data_root = '/global/cscratch1/sd/tang30/ACME_simulations/edison.20181204.noCNT.A_WCYCL1950S_CMIP6_LRtunedHR.ne30_oECv3_ICG'
    fig_root = '/global/homes/q/qingli/work/e3sm_res_cmp/figures/low_res'
    rst_file = data_root+'/run/mpaso.rst.0051-01-01_00000.nc'
    mon_file = data_root+'/archive/ocn/hist/mpaso.hist.am.timeSeriesStatsMonthly.0041-01-01.nc'
else:
    raise EnvironmentError('This script should be executed on either edison or theta')
os.makedirs(fig_root, exist_ok=True)

In [ ]:
# load dataset
f_rst = Dataset(rst_file, 'r')
f_mon = Dataset(mon_file, 'r')

In [ ]:
# read grid information
lon = f_rst.variables['lonCell'][:]*180.0/np.pi
lat = f_rst.variables['latCell'][:]*180.0/np.pi
cellArea = f_rst.variables['areaCell'][:]


### SST (degC)

In [ ]:
# read data
sst = f_mon.variables['timeMonthly_avg_activeTracers_temperature'][0,:,0]
mpaso_sst = MPASOMap(data=sst, lat=lat, lon=lon, cellarea=cellArea, name='SST', units='degC')


In [ ]:
# plot figure
fig = plt.figure()
fig.set_size_inches(6, 5)
mpaso_sst.plot(region='LabSea')

if save_fig:
    figname = fig_root+'/LabSea_sst.png'
    plt.savefig(figname, dpi = 300)

### Mixed Layer Depth (m)

In [ ]:
# read data
mld_d = f_mon.variables['timeMonthly_avg_dThreshMLD'][0,:]
mld_t = f_mon.variables['timeMonthly_avg_tThreshMLD'][0,:]
mpaso_mld_d = MPASOMap(data=mld_d, lat=lat, lon=lon, cellarea=cellArea, name='MLD_rho', units='m')
mpaso_mld_t = MPASOMap(data=mld_t, lat=lat, lon=lon, cellarea=cellArea, name='MLD_temp', units='m')
mpaso_mld_diff = MPASOMap(data=mld_t-mld_d, lat=lat, lon=lon, cellarea=cellArea, name='MLD_temp - MLD_rho', units='m')


In [ ]:
# plot figure
f, axarr = plt.subplots(3)
f.set_size_inches(6, 15)

levels = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 110, 130, 150, 180, 210, 240, 280, 320, 360, 407, 454, 500, 1000, 1500, 2000])
levels_diff = np.linspace(-50, 50, 21)
im0 = mpaso_mld_t.plot(region='LabSea', axis=axarr[0], levels=levels)
im1 = mpaso_mld_d.plot(region='LabSea', axis=axarr[1], levels=levels)
im2 = mpaso_mld_diff.plot(region='LabSea', axis=axarr[2], levels=levels_diff, cmap='RdBu_r')
plt.tight_layout()

if save_fig:
    figname = fig_root+'/LabSea_mld.png'
    plt.savefig(figname, dpi = 300)

### Surface Heat Flux (W m$^{-2}$)

In [ ]:
# read data
shf = f_mon.variables['timeMonthly_avg_sensibleHeatFlux'][0,:]
lhf = f_mon.variables['timeMonthly_avg_latentHeatFlux'][0,:]
swf = f_mon.variables['timeMonthly_avg_shortWaveHeatFlux'][0,:]
lwfu = f_mon.variables['timeMonthly_avg_longWaveHeatFluxUp'][0,:]
lwfd = f_mon.variables['timeMonthly_avg_longWaveHeatFluxDown'][0,:]
lwf = lwfu+lwfd
mpaso_swf = MPASOMap(data=swf, lat=lat, lon=lon, cellarea=cellArea, name='SWF', units=r'W m^{-2}')
mpaso_lwf = MPASOMap(data=lwf, lat=lat, lon=lon, cellarea=cellArea, name='LWF', units=r'W m^{-2}')
mpaso_shf = MPASOMap(data=shf, lat=lat, lon=lon, cellarea=cellArea, name='SHF', units=r'W m^{-2}')
mpaso_lhf = MPASOMap(data=lhf, lat=lat, lon=lon, cellarea=cellArea, name='LHF', units=r'W m^{-2}')


In [ ]:
# plot figure
f, axarr = plt.subplots(2,2)
f.set_size_inches(12, 10)
levels = np.linspace(-400, 400, 81)
im0 = mpaso_swf.plot(region='LabSea', axis=axarr[0,0], levels=levels, cmap='RdBu_r')
im1 = mpaso_lwf.plot(region='LabSea', axis=axarr[0,1], levels=levels, cmap='RdBu_r')
im2 = mpaso_shf.plot(region='LabSea', axis=axarr[1,0], levels=levels, cmap='RdBu_r')
im3 = mpaso_lhf.plot(region='LabSea', axis=axarr[1,1], levels=levels, cmap='RdBu_r')
plt.tight_layout()

if save_fig:
    figname = fig_root+'/LabSea_heatflux.png'
    plt.savefig(figname, dpi = 300)

In [ ]:
mpaso_mld_diff.data.std()